### Check if market data is good or bad

#### Key factors determining if we can trade on with the current market condition:
  *  Market data is accurate: 
      * Deviation of data from two vendor is within certain range / exactly the same
  *  Market condition allows us to carry out the trade: 
      * Liquidity is adequate in the market (Volume)
      * Spread is ideally within certain range so friction cost is acceptable

In [6]:
class OrderBook:
    def __init__(self, vendor, ticker, timestamp, bids, asks):
        """ 
        Compare two order books based on various criteria.
        :param book1: First OrderBook object
        :param book2: Second OrderBook object
        :param price_deviation_threshold: Maximum allowed deviation in prices for accuracy
        :param volume_threshold: Minimum volume for liquidity check
        :param spread_threshold: Maximum spread for acceptable friction cost
        :param jump_threshold: Maximum allowed jump in price relative to the moving average
        :return: Boolean indicating if the market data is good (True) or bad (False)
        """
        self.vendor = vendor
        self.ticker = ticker
        self.timestamp = timestamp 
        self.bids = bids  # List of (price, volume)
        self.asks = asks  # List of (price, volume)

    def get_spread(self):
        best_bid = self.bids[0][0]
        best_ask = self.asks[0][0]
        return best_ask - best_bid
    
    def get_total_volumn(self):
        bid_total_volume =  sum([volumn for _,volumn in self.bids])
        ask_total_volume =  sum([volumn for _,volumn in self.asks])
        return bid_total_volume,ask_total_volume

    def get_depth_volumn(self, level):
        bid_volume = sum([volume for _, volume in self.bids[:level]])
        ask_volume = sum([volume for _, volume in self.asks[:level]])
        return bid_volume, ask_volume
    
    def is_similar(self, other, price_threshold=0.05, volume_threshold=50):
        ''' 
            Use to check if two ordrebook objects are within some similarity threshold
        '''
        for level in range(10):
            self_bid_price, self_bid_volume = self.bids[level]
            other_bid_price, other_bid_volume = other.bids[level]
            self_ask_price, self_ask_volume = self.asks[level]
            other_ask_price, other_ask_volume = other.asks[level]

            # Check price and volume for both bids and asks
            if abs(self_bid_price - other_bid_price) > price_threshold or \
               abs(self_bid_volume - other_bid_volume) > volume_threshold or \
               abs(self_ask_price - other_ask_price) > price_threshold or \
               abs(self_ask_volume - other_ask_volume) > volume_threshold:
                return False
        return True
    
    def check_liquidity(self, volume_threshold):
        total_volume = sum(self.get_total_volumn())
        return total_volume >= volume_threshold
    
    
    def check_balanced_mkt(self, difference_threshold):
        bid_ask_vol_differnence = abs(self.get_total_volumn[0] - self.get_total_volumn[1])
        return bid_ask_vol_differnence <= difference_threshold  

    def check_spread(self, spread_threshold):
        return self.get_spread() <= spread_threshold
    


In [15]:
threshold = {
    'liquidity_threshold' : 1000,
    'spread_threshold' : 0.5,
}
# Function to compare two order books
def compare_order_books(book1, book2,threshold):
    
    # Check two vendor agree with each other
    if not book1.is_similar(book2):
        print("Data Quality Issue ::: Data from two vendors deviating too much")
        return False

    # Check market liquidity
    liquidity_threshold = threshold['liquidity_threshold'] # change for ideal liquidity threshold
    if not book1.check_liquidity(liquidity_threshold):
        print("Market Condition Issue ::: Market Liquidity below threshold")
        return False
    
    # Check spread of market 
    spread_threshold = threshold['spread_threshold']
    if not book1.check_spread(spread_threshold):
        print("Market Condition Issue ::: Market spread is above threshold")
        return False
        
    return True



In [16]:
# Example usage with 10 levels of bids and asks
bids1 = [(150 - i*0.5, 100 + i*10) for i in range(10)]
asks1 = [(151 + i*0.5, 150 + i*10) for i in range(10)]
book1 = OrderBook('Vendor1', 'AAPL', '2023-11-17 09:30:00', bids1, asks1)

bids2 = [(150.5 - i*0.5, 120 + i*15) for i in range(10)]
asks2 = [(152 + i*0.5, 130 + i*15) for i in range(10)]
book2 = OrderBook('Vendor2', 'AAPL', '2023-11-17 09:30:00', bids2, asks2)

comparison_1 = compare_order_books(book1, book2,threshold)
print(comparison_1)

Data Quality Issue ::: Data from two vendors deviating too much
False


In [17]:
bids1 = [(150 - i*0.5, 10 + i*10) for i in range(10)]
asks1 = [(151 + i*0.5, 10 + i*10) for i in range(10)]
book1 = OrderBook('Vendor1', 'AAPL', '2023-11-17 09:30:00', bids1, asks1)

bids2 = [(150 - i*0.5, 10 + i*15) for i in range(10)]
asks2 = [(151 + i*0.5, 10 + i*15) for i in range(10)]
book2 = OrderBook('Vendor2', 'AAPL', '2023-11-17 09:30:00', bids2, asks2)

comparison_2 = compare_order_books(book1, book2,threshold)
print(comparison_2)

Market Condition Issue ::: Market spread is above threshold
False


In [18]:
bids1 = [(150 - i*0.5, 1 + i*10) for i in range(10)]
asks1 = [(150.5 + i*0.5, 1 + i*10) for i in range(10)]
book1 = OrderBook('Vendor1', 'AAPL', '2023-11-17 09:30:00', bids1, asks1)

bids2 = [(150 - i*0.5, 1 + i*15) for i in range(10)]
asks2 = [(150.5 + i*0.5, 1 + i*15) for i in range(10)]
book2 = OrderBook('Vendor2', 'AAPL', '2023-11-17 09:30:00', bids2, asks2)

comparison_3 = compare_order_books(book1, book2,threshold)
print(comparison_3)

Market Condition Issue ::: Market Liquidity below threshold
False
